## 1. Construct datasets in cycle (+for ablation study)

In [1]:
from glue_xlmroberta.dataset_construction import construct_semeval_dataset

/home/alchernyavskiy/anaconda3/envs/py37deberta/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
disco_depth = 2
options = [
    [True, True, True],
    [True, True, False],
    [True, False, False],
    [False, False, False]
]

for part in ['train', 'dev', 'test']: #'train', 'dev', 'test'
    for (use_nucsat, use_rels, use_paths) in options:
        construct_semeval_dataset(
            part = part,
            disco_depth=disco_depth,
            use_rels=use_rels,
            use_nucsat=use_nucsat,
            use_paths=use_paths)

23


100%|█████████████████████████████████████| 1962/1962 [00:00<00:00, 5075.26it/s]


23


100%|█████████████████████████████████████| 1962/1962 [00:00<00:00, 5076.11it/s]


23


...

100%|███████████████████████████████████████| 601/601 [00:00<00:00, 9392.92it/s]


23


100%|██████████████████████████████████████| 601/601 [00:00<00:00, 11008.77it/s]


23


100%|██████████████████████████████████████| 601/601 [00:00<00:00, 12366.99it/s]


In [ ]:
#add no-feats version to train the base model
import json

for part in ['train', 'dev', 'test']:
    fn = f'datasets/deberta_propaganda_classif/{part}_custom_feats=2_lvl2--use_rels=False--use_nucsat=True--use_paths=False_multi_ru.json'

    with open(fn, 'r') as outfile:
        data = json.load(outfile)

    for el in data:
        del el['feature']

    with open(fn.replace('.json', '_nofeat.json'), 'w') as outfile:
        json.dump(data, outfile)

## 2. Save weights for loss

In [5]:
import json
import pickle

In [6]:
with open(f'datasets/deberta_propaganda_classif/train_custom_feats=38_lvl2--use_rels=True--use_nucsat=True--use_paths=False_multi_ru.json', 'r') as outfile:
    data_train = json.load(outfile)

In [7]:
data_train[6]

{'text': 'article241.txt',
 'sentence1': '«Теперь всех, включая этого несчастного [президента Египта Хосни] Мубарака, который просто лизал ноги Западу — всех их обвинили, что они авторитарные лидеры и их надо раздавить. А на их место посадили остервенелых бандитов. Остервенелых! Людоедов. И сказали: это — демократы!», — оценил происходящее политолог.',
 'label': [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  1.0,
  0.0,
  0.0],
 'i': 12,
 'span': [1444, 1755],
 'feature': [0.5,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.25,
  0.0,
  0.0,
  0.0,
  0.25,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.5,
  0.625,
  0.125,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.125,
  0.0,
  0.0,
  0.25,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.125,
  0.375]}

In [8]:
weights =  []
for i in range(len(data_train[0]['label'])):
    label_vals = [] 
    for item in data_train:
        label_vals.append(item['label'][i])
    num_pos = max(sum(label_vals), 1)
    num_neg = len(label_vals) - num_pos
    weights.append(min(70, num_neg / num_pos))

In [9]:
weights

[70,
 35.333333333333336,
 18.04854368932039,
 70,
 69.07142857142857,
 39.875,
 49.30769230769231,
 27.02857142857143,
 21.295454545454547,
 2.8546168958742633,
 13.977099236641221,
 69.07142857142857,
 45.714285714285715,
 70,
 2.060842433697348,
 6.75494071146245,
 70,
 5.475247524752476,
 70,
 27.434782608695652,
 26.25,
 70,
 70]

In [10]:
with open('glue_xlmroberta/pos_weights.pkl', 'wb') as f:
    pickle.dump(weights, f)

In [11]:
with open('glue_xlmroberta/pos_weights.pkl', 'rb') as f:
    weights = pickle.load(f)

In [12]:
labels_list = ['Appeal_to_Authority',
 'Appeal_to_Fear-Prejudice',
 'Appeal_to_Hypocrisy',
 'Appeal_to_Popularity',
 'Appeal_to_Time',
 'Appeal_to_Values',
 'Causal_Oversimplification',
 'Consequential_Oversimplification',
 'Conversation_Killer',
 'Doubt',
 'Exaggeration-Minimisation',
 'False_Dilemma-No_Choice',
 'Flag_Waving',
 'Guilt_by_Association',
 'Loaded_Language',
 'Name_Calling-Labeling',
 'Obfuscation-Vagueness-Confusion',
 'Questioning_the_Reputation',
 'Red_Herring',
 'Repetition',
 'Slogans',
 'Straw_Man',
 'Whataboutism']

In [13]:
list(zip(labels_list, weights))

[('Appeal_to_Authority', 70),
 ('Appeal_to_Fear-Prejudice', 35.333333333333336),
 ('Appeal_to_Hypocrisy', 18.04854368932039),
 ('Appeal_to_Popularity', 70),
 ('Appeal_to_Time', 69.07142857142857),
 ('Appeal_to_Values', 39.875),
 ('Causal_Oversimplification', 49.30769230769231),
 ('Consequential_Oversimplification', 27.02857142857143),
 ('Conversation_Killer', 21.295454545454547),
 ('Doubt', 2.8546168958742633),
 ('Exaggeration-Minimisation', 13.977099236641221),
 ('False_Dilemma-No_Choice', 69.07142857142857),
 ('Flag_Waving', 45.714285714285715),
 ('Guilt_by_Association', 70),
 ('Loaded_Language', 2.060842433697348),
 ('Name_Calling-Labeling', 6.75494071146245),
 ('Obfuscation-Vagueness-Confusion', 70),
 ('Questioning_the_Reputation', 5.475247524752476),
 ('Red_Herring', 70),
 ('Repetition', 27.434782608695652),
 ('Slogans', 26.25),
 ('Straw_Man', 70),
 ('Whataboutism', 70)]

## 3. Train model

In [1]:
from glue_xlmroberta.train_loop import run_train

In [2]:
options = [
    [True, True, True],
     [True, True, False],
     [True, False, False],
    [False, False, False]
]

for (use_nucsat, use_rels, use_paths) in options:
    run_train(level=2,
              lr=2e-5,
              bs=8,
              gac=2,
              n_epochs=45,
              use_rels=use_rels,
              use_nucsat=use_nucsat,
              use_paths=use_paths,
              save_eval_metric='micro_f1',
              device='cuda:0',
              suffix='_ru')